<a href="https://colab.research.google.com/github/kotrying/ML_Theory/blob/main/Back_Propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [9]:
### Back Propagation（誤差逆伝搬）の実装

# 各層の初期化
# クラスからインスタンス化する
middle_layer = MiddleLayer(n_in, n_mid) # （入力層のニューロン数、中間層のニューロン数）を引数へ
output_layer = OutputLayer(n_mid, n_out) # （中間層のニューロン数、出力層のニューロン数）を引数へ

# 学習
# エポック数分の学習を実行
for i in range(epoch):
    # 確率的勾配降下法を実施
    # 入力データの数分の要素を持つインデックス行列を作成
    index_random = np.arange(n_data) # n_data = len(input_data)
    # インデックスをランダムにシャッフル
    np.random.shuffle(index_random)
    
    # 結果の表示用：誤差、入力、出力を保存し、後で学習の結果を確認する
    total_error = 0 # 誤差の初期値：各学習ステップで計算される入力と出力との誤差を加算
    plot_x = [] # 各学習ステップごとに使用した入力
    plot_y = [] # 各学習ステップごとに得られた出力
    
    # インデックス行列から順にインデックスを取り出す
    for idx in index_random:
        # 指定のインデックスに対応するデータをスライス(p50参照)で取り出す（バッチサイズは１）
        x = input_data[idx:idx+1]  # 入力：指定のインデックスにおける入力データの値
        t = correct_data[idx:idx+1]  # 正解：指定のインデックスにおける正解データの値
        
        # 順伝播
        middle_layer.forward(x.reshape(1, 1)) # 入力を行列（行数１、列数１）に変換
        output_layer.forward(middle_layer.y) # 中間層→出力層

        # 逆伝播
        output_layer.backward(t.reshape(1, 1))  # 正解を行列（行数１、列数１）に変換
        middle_layer.backward(output_layer.grad_x) # 出力層→中間層
        
        # 重みとバイアスの更新
        middle_layer.update(eta)
        output_layer.update(eta)

In [57]:
# 中間層
class MiddleLayer:
    def __init__(self, n_upper, n, wb_width = 0.01):  # 初期設定
        self.w = wb_width * np.random.randn(n_upper, n)  # 重み（行列）
        self.b = wb_width * np.random.randn(n)  # バイアス（ベクトル）

    def forward(self, x):  # 順伝播
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = 1/(1+np.exp(-u))  # シグモイド関数
    
    def backward(self, grad_y):  # 逆伝播
        delta = grad_y * (1-self.y)*self.y  # シグモイド関数の微分
        
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        
        self.grad_x = np.dot(delta, self.w.T) 
        
    def update(self, eta):  # 重みとバイアスの更新
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

# 出力層
class OutputLayer:
    def __init__(self, n_upper, n, wb_width = 0.01):  # 初期設定
        self.w = wb_width * np.random.randn(n_upper, n)  # 重み（行列）
        self.b = wb_width * np.random.randn(n)  # バイアス（ベクトル）
    
    def forward(self, x):  # 順伝播
        self.x = x
        u = np.dot(x, self.w) + self.b
        self.y = u  # 恒等関数
    
    def backward(self, t):  # 逆伝播
        delta = self.y - t
        
        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)
        
        self.grad_x = np.dot(delta, self.w.T) 

    def update(self, eta):  # 重みとバイアスの更新
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b

# Back Propagation（誤差逆伝搬）の実装
def back_propagation(input_data: np.ndarray, correct_data: np.ndarray, n_in: int, n_mid: int, n_out: int, epoch: int, eta: int):

    np.random.seed(181045)

    # 各層の初期化
    # クラスからインスタンス化する
    middle_layer = MiddleLayer(n_in, n_mid) # （入力層のニューロン数、中間層のニューロン数）を引数へ
    output_layer = OutputLayer(n_mid, n_out) # （中間層のニューロン数、出力層のニューロン数）を引数へ

    n_data = len(input_data)

    # 学習
    # エポック数分の学習を実行
    for i in range(epoch):
        # 確率的勾配降下法を実施
        # 入力データの数分の要素を持つインデックス行列を作成
        index_random = np.arange(n_data)
        # インデックスをランダムにシャッフル
        np.random.shuffle(index_random)
        
        # 結果の表示用：誤差、入力、出力を保存し、後で学習の結果を確認する
        total_error = 0 # 誤差の初期値：各学習ステップで計算される入力と出力との誤差を加算
        plot_y = np.zeros((n_data, ), dtype=np.float32) # 各学習ステップごとに得られた出力
        
        # インデックス行列から順にインデックスを取り出す
        for idx in index_random:
            # 指定のインデックスに対応するデータをスライス(p50参照)で取り出す（バッチサイズは１）
            x = input_data[idx:idx+1]  # 入力：指定のインデックスにおける入力データの値
            t = correct_data[idx:idx+1]  # 正解：指定のインデックスにおける正解データの値
            
            # 順伝播
            middle_layer.forward(x.reshape(1, 1)) # 入力を行列（行数１、列数１）に変換
            output_layer.forward(middle_layer.y) # 中間層→出力層

            # 逆伝播
            output_layer.backward(t.reshape(1, 1))  # 正解を行列（行数１、列数１）に変換
            middle_layer.backward(output_layer.grad_x) # 出力層→中間層
            
            # 重みとバイアスの更新
            middle_layer.update(eta)
            output_layer.update(eta)

            if i%100==0:
                plot_y[idx] = output_layer.y.reshape(-1)
        if i%100==0:
            print(f'- {i+1}回目の出力 : {plot_y}')

In [63]:
# 入力データ
input_data = np.array([1,2,3,4,5])
# 正解データ
correct_data = np.array([2,4,6,8,10])
# 実行
back_propagation(input_data, correct_data, n_in=1, n_mid=3, n_out=1, epoch=1501, eta=0.1)

- 1回目の出力 : [0.71048784 0.01762506 5.0574017  0.93565327 2.5873468 ]
- 101回目の出力 : [2.275919  3.378271  6.5474505 7.7531214 9.658143 ]
- 201回目の出力 : [2.2071044 3.8314712 6.5546637 7.774943  9.593967 ]
- 301回目の出力 : [2.3319213 3.3205817 6.295534  8.09365   9.657005 ]
- 401回目の出力 : [2.2528434 3.6546729 6.4118514 7.913717  9.921646 ]
- 501回目の出力 : [2.2949748 3.7310224 5.8647985 7.8433623 9.980521 ]
- 601回目の出力 : [ 2.1274688  3.8599913  6.0777335  8.070919  10.32872  ]
- 701回目の出力 : [2.045355 4.121908 5.59853  8.539867 9.631587]
- 801回目の出力 : [2.0145957 3.9454331 5.873819  8.216927  9.877725 ]
- 901回目の出力 : [1.9999776 4.0114007 5.9781847 8.005793  9.980517 ]
- 1001回目の出力 : [1.9990685 3.9997993 5.94052   8.065308  9.977423 ]
- 1101回目の出力 : [ 1.9999874  4.002497   5.998506   7.995465  10.004804 ]
- 1201回目の出力 : [1.9998585 4.001294  6.001886  8.003889  9.99597  ]
- 1301回目の出力 : [1.9996318 3.9999554 6.0005035 8.003841  9.994724 ]
- 1401回目の出力 : [2.0000238 4.000538  5.9985824 8.001151  9.999988 ]
- 1501回目の出力 